In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
device = "cuda"
import pickle
import torch
import numpy as np
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans

In [ ]:
#fm = Path("/data/natsuki/training116/00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/network-snapshot-011289.pkl")
fm = Path("/data/natsuki/training116/00023-white_yc05_yw04-mirror-auto4-gamma10-noaug/network-snapshot-021800.pkl")
assert fm.is_file()
with open(fm, 'rb') as f:
    G = pickle.load(f)['G_ema'].to(device)
def mapping(seed=1, psi=1):
    label = torch.zeros([1, G.c_dim], device=device)
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    w = G.mapping(z, label, truncation_psi=psi)
    return w
def synthesis(w, synth=True):
    if synth:
        synth_image = G.synthesis(w, noise_mode='const')
    else:
        synth_image = w
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    img = Image.fromarray(synth_image, 'RGB')
    return img

In [ ]:
batch = 32
sample_num = 6 * 10**4
cluster_num = 64
label = torch.zeros([1, G.c_dim], device=device)
assert sample_num % batch == 0
ws = list()
for j in tqdm(range(sample_num//batch)):
    ws += list(G.mapping(torch.from_numpy(np.array([np.random.RandomState(batch*j+i).randn(G.z_dim) for i in range(batch)])).to(device), label).cpu().numpy()[:,0,:])
ws = np.array(ws)
print(ws.shape)

In [ ]:
km = KMeans(cluster_num, init="random")
km.fit(ws)
print(km.cluster_centers_.shape)

In [ ]:
dst = Image.new("RGB", (4096, 4096))
for i in range(8):
    for j in range(8):
        tmp = synthesis(torch.from_numpy(km.cluster_centers_[None,None,8*i+j]).repeat((1,16,1)).to(device))
        font = ImageFont.truetype("/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf", 50)
        ImageDraw.Draw(tmp).text((0,0), f"{8*i+j}", (0,0,0), font)
        dst.paste(tmp, (512*j, 512*i))
dst